# Import Libraries

In [3]:
!pip install sentence-transformers faiss-cpu sentencepiece


from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import faiss
import numpy as np


# Knowlwgw Base Creation

In [13]:
kb_chunks = [
    "Some people forget that love is tucking you in and kissing you 'Good night' no matter how young or old you are.",
    "Some people don't remember that love is listening and laughing and asking questions no matter what your age.",
    "Few recognize that love is commitment, responsibility — no fun at all unless love is you and me."
]
kb_chunks.append("The poem 'Love Is' by Nikki Giovanni was published in 1997 in her collection titled 'Love Poems'.")

# Embedding And Indexing

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')
kb_embeddings = model.encode(kb_chunks)


dimension = kb_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(kb_embeddings))


# Retrival Function

In [15]:
def retrieve_chunks(query, top_k=2):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [kb_chunks[i] for i in indices[0]]

# Generation With Context

In [16]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

def generate_answer(query):
    context = retrieve_chunks(query)
    prompt = f"""You are a poetic assistant. Use the context below to answer the question in a thoughtful, complete sentence. 
If the answer is not in the context, respond clearly with: "The answer is not in the context."

Context:
{chr(10).join(context)}

Question: {query}
Answer:"""
    inputs = tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True)
    output = generator.generate(**inputs, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# test cases

In [17]:
print ("Q1: ", generate_answer("What is love?"))

print("Q2:", generate_answer("What are some actions that show love?"))


print("Q3:", generate_answer("What year was this poem written?"))


print("Q4:", generate_answer("How does the poem contrast emotional and practical aspects of love?"))

Q1:  commitment, responsibility
Q2: listening and laughing and asking questions no matter what your age
Q3: 1997
Q4: Few recognize that love is commitment, responsibility — no fun at all unless love is you and me.
